In [1]:
import pandas as pd
import numpy as np

In [2]:
NKE = pd.read_csv("NKE_pred.csv")  
JPM = pd.read_csv("JPM_pred.csv")  
SBUX = pd.read_csv("SBUX_pred.csv")  
BRK_B = pd.read_csv("BRK-B_pred.csv")  
RRC = pd.read_csv("RRC_pred.csv")  
CAR = pd.read_csv("CAR_pred.csv")  
COST = pd.read_csv("COST_pred.csv")  

In [11]:
df = pd.merge(NKE, JPM,how='left', on='timestamp')
df = pd.merge(df, SBUX,how='left', on='timestamp')
df = pd.merge(df, BRK_B,how='left', on='timestamp')
df = pd.merge(df, RRC,how='left', on='timestamp')
df = pd.merge(df, CAR,how='left', on='timestamp')
df = pd.merge(df, COST,how='left', on='timestamp')

In [12]:
df

,timestamp,NKE real value,NKE prediction,JPM real value,JPM prediction,SBUX real value,SBUX prediction,BRK-B real value,BRK-B prediction,RRC real value,RRC prediction,CAR real value,CAR prediction,COST real value,COST prediction
0,2019-01-02,74.06,74.190105,99.31,97.644929,64.32,64.422021,202.80,203.158976,9.96,9.624571,22.71,22.840629,204.76,204.986283
1,2019-01-03,72.75,74.240245,97.11,97.669864,61.53,64.444050,191.66,202.143059,9.95,9.679453,21.78,23.207044,200.42,206.270561
2,2019-01-04,74.65,74.290418,100.69,97.694806,63.57,64.466087,195.20,201.132221,10.69,9.734648,23.13,23.579337,206.24,207.562886
3,2019-01-07,75.72,74.340625,100.76,97.719754,63.57,64.488131,196.91,200.126438,11.13,9.790157,23.93,23.957602,207.00,208.863308
4,2019-01-08,76.73,74.390866,100.57,97.744709,63.62,64.510182,196.31,199.125685,11.00,9.845984,24.11,24.341935,208.55,210.171877
5,2019-01-09,76.59,74.441141,100.40,97.769669,63.88,64.532242,196.37,198.129936,11.60,9.902128,25.06,24.732434,209.90,211.488644
6,2019-01-10,76.42,74.491450,100.39,97.794637,64.19,64.554308,196.58,197.139167,11.44,9.958593,25.57,25.129197,210.64,212.813661
7,2019-01-11,76.04,74.541793,99.91,97.819610,63.73,64.576382,196.29,196.153352,11.50,10.015380,26.01,25.532325,210.51,214.146980
8,2019-01-14,76.09,74.592170,100.94,97.844590,63.37,64.598464,195.78,195.172466,11.87,10.072490,26.04,25.941921,209.00,215.488652
9,2019-01-15,77.88,74.642581,101.68,97.869577,64.08,64.620553,196.94,194.196486,11.74,10.129926,25.71,26.358087,211.03,216.838731


In [13]:
df.to_csv('ARIMA+GARCH_preds_3months.csv')

# Baseline MPT

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas_datareader import data as pdr
import fix_yahoo_finance
import datetime
import os
import cvxpy as cp

In [15]:
tickers = ['NKE', 'JPM', 'SBUX', 'BRK-B', 'RRC', 'CAR', 'COST']
risk_free_rate = 0.026 # 1-year T-bill on 1 Jan 2019

In [16]:
def get_adj_closing_prices(tickers, start_date, end_date):
    all_prices = {}
    for ticker in tickers:
        prices = pdr.get_data_yahoo(ticker, 
            start=start_date,
            end=end_date
        )
        all_prices[ticker] = prices["Adj Close"]
    
    return pd.DataFrame(all_prices)

In [22]:
def compute_expected_daily_returns(df):
    return np.array(np.mean(df.T, axis=1))

In [24]:
def compute_daily_covariance_matrix(df): 
    return np.cov(df.T)

In [27]:
def compute_portfolio_expected_return(return_vector, weights):
    """Computes expected return of a portfolio given the asset returns and the asset weights in the portfolio."""
    return return_vector.T.dot(weights)

def compute_portfolio_variance(covariance_matrix, weights):
    """Computes the variance of a portfolio given the asset covariance matrix and the asset weights in the portfolio."""
    return weights.T.dot(covariance_matrix).dot(weights)

def compute_portfolio_std(covariance_matrix, weights):
    """Computes the standard deviation of a portfolio given the asset covariance matrix and the asset weights in the portfolio.""" 
    return np.sqrt(compute_portfolio_variance(covariance_matrix, weights))

In [28]:
 def min_risk_portfolio(expected_returns, covariance_matrix, target_return):
    """Computes the weights of a minimum variance portfolio for a given expected return.
    The portfolio is fully vested (weights sum to 1) and no shorting is allowed.
    
    Args:
        expected_returns (np.array): Expected returns for n assets.
        covariance_matrix (np.array): n x n Covariance matrix of asset returns.
        target_return (float): Expected return target for the portfolio
        
    Returns:
        np.array of length n with the weights of each asset in the portfolio
        OR
        None if no feasible portfolio exists.
    """
    n = expected_returns.shape[0]        #数组的个数
    
    w = cp.Variable(n)                         # Portfolio allocation vector   ?cp.Variable: The optimization variables in a problem.定义问题的变量
    ret = expected_returns.T * w
    risk = cp.quad_form(w,covariance_matrix)  #就是组合方差
    target_ret = cp.Parameter()             #定义问题的参数
    target_ret.value = target_return
    prob = cp.Problem(cp.Minimize(risk),          # Restricting to long-only portfolio
                   [ret == target_ret, # match target_return
                   cp.sum(w) == 1,    # sum of weights in portfolios sum to 1.
                   w >= 0])
    prob.solve()
    
    if prob.status == 'optimal':
        return w.value
    else:
        return None

In [29]:
def efficient_frontier(expected_returns, covariance_matrix):
    """Construct efficient frontier portfolios by a line sweep of target returns.
    
    Args:
        expected_returns (np.array): Expected returns for n assets.
        covariance_matrix (np.array): n x n Covariance matrix of asset returns.
        
    Returns:
        List of np.arrays. Each numpy array in the list represents a portfolio on the efficient frontier.
    
    """
    min_return = np.min(expected_returns)
    max_return = np.max(expected_returns)
    
    target_returns = np.linspace(min_return,max_return, num=200)
    
    portfolio_weights = []
    
    for tr in target_returns:
        result = min_risk_portfolio(expected_returns, covariance_matrix, tr)
        # only add results if optimization was successful
        if result is not None:
            weights = result
            portfolio_weights.append(weights)

    return portfolio_weights

In [31]:
def min_variance_portfolio(covariance_matrix, portfolio_weights):
    """Select minimum variance portfolio from a list of portfolios.
    
    Args:
        covariance_matrix (np.array): n x n Covariance matrix of asset returns.
        portfolio_weights (list of np.array): Each numpy array in the list represents a portfolio.
        
    Returns:
        np.array with weights of portfolio that has the smallest variance.
    """
    portfolio_variances = []

    for weight in portfolio_weights:  
        variance = compute_portfolio_variance(covariance_matrix, weight)
        portfolio_variances.append(variance) 
        
    index = np.argmin(portfolio_variances)   #Returns the indices of the minimum values along an axis.
    
    return portfolio_weights[index]

In [34]:
def max_sharpe_portfolio(expected_returns, covariance_matrix, risk_free_rate, portfolio_weights):
    """Select max sharpe portfolio from a list of portfolios.
    
    Args:
        expected_returns (np.array): Expected returns for n assets.
        covariance_matrix (np.array): n x n Covariance matrix of asset returns.
        risk_free_rate (float): Risk free rate.
        portfolio_weights (list of np.array): Each numpy array in the list represents a portfolio.
        
    Returns:
        np.array with weights of portfolio that maximizes sharpe ratio.  
    """
    sharpe_ratios = []

    for weight in portfolio_weights:  
        mean = compute_portfolio_expected_return(expected_returns, weight)
        std = compute_portfolio_std(covariance_matrix, weight)
        sharpe_ratio = (mean - risk_free_rate)/std
        sharpe_ratios.append(sharpe_ratio) 
        
    index = np.argmax(sharpe_ratios)
    
    return sharpe_ratios[index], portfolio_weights[index]

In [ ]:
Prices2019 = get_adj_closing_prices(tickers, datetime.datetime(2018, 12, 31), datetime.datetime(2019, 3, 29))

# Jan

In [157]:
# get daily price data from 1998 to 2019 Jan
prices = get_adj_closing_prices(tickers, datetime.datetime(1998, 12, 31), datetime.datetime(2018, 12, 31))

In [158]:
prices.tail()

,NKE,JPM,SBUX,BRK-B,RRC,CAR,COST
Date,,,,,,,
2018-12-24,67.925240,90.703705,60.245586,187.759995,9.402572,22.379999,189.478210
2018-12-26,72.822639,94.464165,62.752502,197.570007,10.400722,24.049999,198.094940
2018-12-27,73.480942,95.527321,62.871880,200.119995,10.171146,23.980000,200.538345
2018-12-28,73.151787,95.320602,63.060890,202.080002,9.622165,23.549999,201.495743
2018-12-31,73.949738,96.098289,64.065651,204.179993,9.552294,22.480000,203.161255


In [159]:
JanStart = Prices2019['2018-12-31':'2018-12-31']
JanEnd = Prices2019['2019-01-31':'2019-01-31']

In [160]:
daily_returns = prices.pct_change().dropna()

In [161]:
exp_returns_daily = compute_expected_daily_returns(daily_returns)
# convert average daily returns to annualized returns
exp_returns_monthly = np.power(1 + exp_returns_daily, 20) - 1

In [162]:
cov_daily = compute_daily_covariance_matrix(daily_returns)
cov_matrix_monthly = cov_daily * 20 # monthly: multiply with number of trading days in a month

In [163]:
print("Monthly Covariance Matrix")
print(' ' * 7, end="")   # 不换行输出
for tick in tickers:
    print("% 7s" % tick, end='')
print('\n', end="") #\n表示换行
print(' ' * 5, end="")
print('-' * 72)
for i, row_ticker in enumerate(tickers): #enumerate同时列出数据下标和数据
    print('%-4s | ' % row_ticker, end='')
    for j, col_ticker in enumerate(tickers):
        print("% 6.4f" % cov_matrix_annual[i,j], end='')
    print('\n', end='')

Monthly Covariance Matrix
           NKE    JPM   SBUX  BRK-B    RRC    CAR   COST
     ------------------------------------------------------------------------
NKE  |  0.0983 0.0442 0.0356 0.0211 0.0317 0.0527 0.0319
JPM  |  0.0442 0.1546 0.0559 0.0382 0.0498 0.0867 0.0436
SBUX |  0.0356 0.0559 0.1284 0.0237 0.0325 0.0576 0.0383
BRK-B |  0.0211 0.0382 0.0237 0.0514 0.0257 0.0389 0.0178
RRC  |  0.0317 0.0498 0.0325 0.0257 0.2689 0.0633 0.0228
CAR  |  0.0527 0.0867 0.0576 0.0389 0.0633 0.4451 0.0412
COST |  0.0319 0.0436 0.0383 0.0178 0.0228 0.0412 0.0837


In [164]:
portfolio_weights = efficient_frontier(exp_returns_monthly, cov_matrix_monthly)

In [165]:
min_variance_weights = min_variance_portfolio(cov_matrix_monthly, portfolio_weights)
min_variance_expected_return = compute_portfolio_expected_return(exp_returns_monthly, min_variance_weights)
min_variance_std = compute_portfolio_std(cov_matrix_monthly, min_variance_weights)

In [166]:
for tick, x in zip(tickers, min_variance_weights):
    print('Weight %-4s %5.2f%%' % (tick, (x + 1e-10) * 100 ))

Weight NKE  14.61%
Weight JPM   0.00%
Weight SBUX  6.84%
Weight BRK-B 51.86%
Weight RRC   4.08%
Weight CAR   0.00%
Weight COST 22.60%


In [167]:
JanReturn = np.dot(JanEnd, (min_variance_weights/JanStart).T)
JanReturnRate = (JanReturn - 1)/1
print('Jan Return rate of min_variance: %5.2f%% ' % ((JanReturnRate + 1e-10) * 100) )

Jan Return rate of min_variance:  4.10% 


In [168]:
max_sharpe_ratio, max_sharpe_weights = max_sharpe_portfolio(exp_returns_monthly, cov_matrix_monthly, risk_free_rate, portfolio_weights)
max_sharpe_expected_return = compute_portfolio_expected_return(exp_returns_monthly, max_sharpe_weights)
max_sharpe_std = compute_portfolio_std(cov_matrix_monthly, max_sharpe_weights)

In [169]:
for tick, x in zip(tickers, max_sharpe_weights):
    print('Weight %-4s %5.2f%%' % (tick, (x + 1e-10) * 100 ))

Weight NKE  99.96%
Weight JPM  -0.00%
Weight SBUX  0.01%
Weight BRK-B  0.00%
Weight RRC   0.01%
Weight CAR   0.01%
Weight COST -0.01%


In [171]:
JanReturn = np.dot(JanEnd, (max_sharpe_weights/JanStart).T)
JanReturnRate = (JanReturn - 1)/1
print('Jan Return rate of max_sharpe: %5.2f%% ' % ((JanReturnRate + 1e-10) * 100) )

Jan Return rate of max_sharpe: 10.41% 


# Feb

In [125]:
# get daily price data from 1998 to 2019 Feb
prices = get_adj_closing_prices(tickers, datetime.datetime(1998, 12, 31), datetime.datetime(2018, 1, 30))

In [126]:
prices.tail()

,NKE,JPM,SBUX,BRK-B,RRC,CAR,COST
Date,,,,,,,
2018-01-25,66.781830,111.925507,58.870888,215.100006,15.209841,45.060001,195.366898
2018-01-26,67.107300,112.525284,56.381886,217.190002,15.189985,46.450001,196.314407
2018-01-29,66.653603,112.409195,55.438778,215.460007,14.981496,46.410000,195.732086
2018-01-30,66.407028,111.354759,55.604061,214.410004,14.524802,45.799999,195.327438
2018-01-31,67.284828,111.896484,55.234604,214.380005,14.147534,44.959999,192.327118


In [175]:
FebStart = Prices2019['2019-02-01':'2019-02-01']
FebEnd = Prices2019['2019-02-28':'2019-02-28']

In [134]:
daily_returns = prices.pct_change().dropna()

In [135]:
exp_returns_daily = compute_expected_daily_returns(daily_returns)
# convert average daily returns to annualized returns
exp_returns_monthly = np.power(1 + exp_returns_daily, 20) - 1

In [136]:
cov_daily = compute_daily_covariance_matrix(daily_returns)
cov_matrix_monthly = cov_daily * 20 # monthly: multiply with number of trading days in a month

In [137]:
print("Monthly Covariance Matrix")
print(' ' * 7, end="")   # 不换行输出
for tick in tickers:
    print("% 7s" % tick, end='')
print('\n', end="") #\n表示换行
print(' ' * 5, end="")
print('-' * 72)
for i, row_ticker in enumerate(tickers): #enumerate同时列出数据下标和数据
    print('%-4s | ' % row_ticker, end='')
    for j, col_ticker in enumerate(tickers):
        print("% 6.4f" % cov_matrix_annual[i,j], end='')
    print('\n', end='')

Monthly Covariance Matrix
           NKE    JPM   SBUX  BRK-B    RRC    CAR   COST
     ------------------------------------------------------------------------
NKE  |  0.0983 0.0442 0.0356 0.0211 0.0317 0.0527 0.0319
JPM  |  0.0442 0.1546 0.0559 0.0382 0.0498 0.0867 0.0436
SBUX |  0.0356 0.0559 0.1284 0.0237 0.0325 0.0576 0.0383
BRK-B |  0.0211 0.0382 0.0237 0.0514 0.0257 0.0389 0.0178
RRC  |  0.0317 0.0498 0.0325 0.0257 0.2689 0.0633 0.0228
CAR  |  0.0527 0.0867 0.0576 0.0389 0.0633 0.4451 0.0412
COST |  0.0319 0.0436 0.0383 0.0178 0.0228 0.0412 0.0837


In [138]:
portfolio_weights = efficient_frontier(exp_returns_monthly, cov_matrix_monthly)

In [139]:
min_variance_weights = min_variance_portfolio(cov_matrix_monthly, portfolio_weights)
min_variance_expected_return = compute_portfolio_expected_return(exp_returns_monthly, min_variance_weights)
min_variance_std = compute_portfolio_std(cov_matrix_monthly, min_variance_weights)

In [140]:
for tick, x in zip(tickers, min_variance_weights):
    print('Weight %-4s %5.2f%%' % (tick, (x + 1e-10) * 100 ))

Weight NKE  14.97%
Weight JPM   0.00%
Weight SBUX  6.32%
Weight BRK-B 52.16%
Weight RRC   4.31%
Weight CAR   0.00%
Weight COST 22.24%


In [154]:
FebReturn = np.dot(FebEnd, (min_variance_weights/FebStart).T)
FebReturnRate = (FebReturn - 1)/1
print('Jan Return rate of min_variance: %5.2f%% ' % ((FebReturnRate + 1e-10) * 100) )

Jan Return rate of min_variance: -0.10% 


In [142]:
max_sharpe_ratio, max_sharpe_weights = max_sharpe_portfolio(exp_returns_monthly, cov_matrix_monthly, risk_free_rate, portfolio_weights)
max_sharpe_expected_return = compute_portfolio_expected_return(exp_returns_monthly, max_sharpe_weights)
max_sharpe_std = compute_portfolio_std(cov_matrix_monthly, max_sharpe_weights)

In [143]:
for tick, x in zip(tickers, max_sharpe_weights):
    print('Weight %-4s %5.2f%%' % (tick, (x + 1e-10) * 100 ))

Weight NKE  100.00%
Weight JPM   0.00%
Weight SBUX  0.00%
Weight BRK-B  0.00%
Weight RRC   0.00%
Weight CAR   0.00%
Weight COST  0.00%


In [156]:
FebReturn = np.dot(FebEnd,(max_sharpe_weights/FebStart).T)
FebReturnRate = (FebReturn - 1)/1
print('Jan Return rate of max_sharpe: %5.2f%% ' % ((FebReturnRate + 1e-10) * 100) )

Jan Return rate of max_sharpe:  5.18% 


# Mar

In [174]:
# get daily price data from 1998 to 2019 Mar
prices = get_adj_closing_prices(tickers, datetime.datetime(1998, 12, 31), datetime.datetime(2018, 2, 27))
prices.tail()

,NKE,JPM,SBUX,BRK-B,RRC,CAR,COST
Date,,,,,,,
2018-02-22,66.209763,111.229004,54.155861,200.729996,12.906524,44.200001,184.647949
2018-02-23,67.225655,113.482994,54.879238,202.759995,13.353288,45.830002,187.834427
2018-02-26,68.695221,114.895355,55.886105,210.619995,13.452568,46.310001,190.803207
2018-02-27,67.097435,113.531349,55.299583,209.660004,13.065372,45.240002,188.121414
2018-02-28,66.111145,111.732040,55.817677,207.199997,13.194438,45.180000,188.913071


In [176]:
MarStart = Prices2019['2019-03-01':'2019-03-01']
MarEnd = Prices2019['2019-03-29':'2019-03-29']

In [178]:
MarEnd

,NKE,JPM,SBUX,BRK-B,RRC,CAR,COST
Date,,,,,,,
2019-03-29,84.209999,100.461288,74.339996,200.889999,11.24,34.860001,242.139999


In [179]:
daily_returns = prices.pct_change().dropna()

In [180]:
exp_returns_daily = compute_expected_daily_returns(daily_returns)
# convert average daily returns to annualized returns
exp_returns_monthly = np.power(1 + exp_returns_daily, 20) - 1

In [181]:
cov_daily = compute_daily_covariance_matrix(daily_returns)
cov_matrix_monthly = cov_daily * 20 # monthly: multiply with number of trading days in a month

In [182]:
print("Monthly Covariance Matrix")
print(' ' * 7, end="")   # 不换行输出
for tick in tickers:
    print("% 7s" % tick, end='')
print('\n', end="") #\n表示换行
print(' ' * 5, end="")
print('-' * 72)
for i, row_ticker in enumerate(tickers): #enumerate同时列出数据下标和数据
    print('%-4s | ' % row_ticker, end='')
    for j, col_ticker in enumerate(tickers):
        print("% 6.4f" % cov_matrix_annual[i,j], end='')
    print('\n', end='')

Monthly Covariance Matrix
           NKE    JPM   SBUX  BRK-B    RRC    CAR   COST
     ------------------------------------------------------------------------
NKE  |  0.0983 0.0442 0.0356 0.0211 0.0317 0.0527 0.0319
JPM  |  0.0442 0.1546 0.0559 0.0382 0.0498 0.0867 0.0436
SBUX |  0.0356 0.0559 0.1284 0.0237 0.0325 0.0576 0.0383
BRK-B |  0.0211 0.0382 0.0237 0.0514 0.0257 0.0389 0.0178
RRC  |  0.0317 0.0498 0.0325 0.0257 0.2689 0.0633 0.0228
CAR  |  0.0527 0.0867 0.0576 0.0389 0.0633 0.4451 0.0412
COST |  0.0319 0.0436 0.0383 0.0178 0.0228 0.0412 0.0837


In [183]:
portfolio_weights = efficient_frontier(exp_returns_monthly, cov_matrix_monthly)

In [184]:
min_variance_weights = min_variance_portfolio(cov_matrix_monthly, portfolio_weights)
min_variance_expected_return = compute_portfolio_expected_return(exp_returns_monthly, min_variance_weights)
min_variance_std = compute_portfolio_std(cov_matrix_monthly, min_variance_weights)

In [185]:
for tick, x in zip(tickers, min_variance_weights):
    print('Weight %-4s %5.2f%%' % (tick, (x + 1e-10) * 100 ))

Weight NKE  15.13%
Weight JPM   0.00%
Weight SBUX  6.61%
Weight BRK-B 51.67%
Weight RRC   4.37%
Weight CAR   0.00%
Weight COST 22.23%


In [191]:
MarReturn = np.dot(MarEnd, (min_variance_weights/MarStart).T)
MarReturnRate = (MarReturn - 1)/1
print('Mar Return rate of min_variance: %5.2f%% ' % ((MarReturnRate + 1e-10) * 100) )

Mar Return rate of min_variance:  1.67% 


In [187]:
max_sharpe_ratio, max_sharpe_weights = max_sharpe_portfolio(exp_returns_monthly, cov_matrix_monthly, risk_free_rate, portfolio_weights)
max_sharpe_expected_return = compute_portfolio_expected_return(exp_returns_monthly, max_sharpe_weights)
max_sharpe_std = compute_portfolio_std(cov_matrix_monthly, max_sharpe_weights)

In [188]:
for tick, x in zip(tickers, max_sharpe_weights):
    print('Weight %-4s %5.2f%%' % (tick, (x + 1e-10) * 100 ))

Weight NKE  100.00%
Weight JPM   0.00%
Weight SBUX  0.00%
Weight BRK-B  0.00%
Weight RRC   0.00%
Weight CAR   0.00%
Weight COST  0.00%


In [190]:
MarReturn = np.dot(MarEnd,(max_sharpe_weights/MarStart).T)
MarReturnRate = (MarReturn - 1)/1
print('Mar Return rate of max_sharpe: %5.2f%% ' % ((MarReturnRate + 1e-10) * 100) )

Mar Return rate of max_sharpe: -3.38% 
